In [5]:
import pyspark
from pyspark.sql import SparkSession

# MinIO credentials
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"

# Create Spark conf for Iceberg + REST + MinIO (using AWS SDK via S3FileIO)
conf = (
    pyspark.SparkConf()
        .setAppName("iceberg-rest-test")

        # Required JARs for Iceberg and AWS SDK
        .set("spark.jars.packages",
             "org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.0,"
             "software.amazon.awssdk:bundle:2.17.178,"
             "software.amazon.awssdk:url-connection-client:2.17.178")

        # Iceberg extensions
        .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

        # REST catalog setup
        .set("spark.sql.catalog.rest", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.rest.type", "rest")
        .set("spark.sql.catalog.rest.uri", "http://iceberg-rest:8181")
        .set("spark.sql.catalog.rest.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .set("spark.sql.catalog.rest.warehouse", "s3://warehouse")
        .set("spark.sql.catalog.rest.s3.endpoint", "http://minio:9000")
        .set("spark.sql.catalog.rest.s3.access-key-id", MINIO_ACCESS_KEY)
        .set("spark.sql.catalog.rest.s3.secret-access-key", MINIO_SECRET_KEY)

        # Optional: make "rest" the default catalog
        .set("spark.sql.defaultCatalog", "rest")
)

# Start Spark session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Optional: set log level
spark.sparkContext.setLogLevel("INFO")

print("✅ Spark session started")

# Create namespace if not exists
spark.sql("CREATE NAMESPACE IF NOT EXISTS rest.default")

# Create a simple table
spark.sql("""
CREATE TABLE IF NOT EXISTS rest.default.sample_table (
  id INT,
  name STRING
)
USING iceberg
""")

# Insert a sample row
spark.sql("INSERT INTO rest.default.sample_table VALUES (1, 'akshay')")

# Query and show the result
df = spark.sql("SELECT * FROM rest.default.sample_table")
df.show()

25/04/23 15:04:50 INFO SparkContext: Running Spark version 3.3.0
25/04/23 15:04:50 INFO ResourceUtils: ==============================================================
25/04/23 15:04:50 INFO ResourceUtils: No custom resources configured for spark.driver.
25/04/23 15:04:50 INFO ResourceUtils: ==============================================================
25/04/23 15:04:50 INFO SparkContext: Submitted application: iceberg-rest-test
25/04/23 15:04:50 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/04/23 15:04:50 INFO ResourceProfile: Limiting resource is cpu
25/04/23 15:04:50 INFO ResourceProfileManager: Added ResourceProfile id: 0
25/04/23 15:04:50 INFO SecurityManager: Changing view acls to: jovyan
25/04/23 15:04:50 INFO SecurityManager:

Py4JJavaError: An error occurred while calling o144.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:162)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:109)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:93)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:176)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:285)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:219)
	at org.apache.iceberg.rest.HTTPClient.post(HTTPClient.java:330)
	at o

In [2]:
# Create namespace
spark.sql("CREATE NAMESPACE IF NOT EXISTS rest.default")




25/04/23 15:01:11 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/04/23 15:01:11 INFO SharedState: Warehouse path is 'file:/home/jovyan/work/spark-warehouse'.
25/04/23 15:01:11 INFO CatalogUtil: Loading custom FileIO implementation: org.apache.iceberg.aws.s3.S3FileIO


DataFrame[]

In [3]:
# Create table
spark.sql("""
CREATE TABLE rest.default.sample_table (
  id INT,
  name STRING
)
USING iceberg
""")



Py4JJavaError: An error occurred while calling o45.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:162)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:109)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:93)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:176)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:285)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:219)
	at org.apache.iceberg.rest.HTTPClient.post(HTTPClient.java:330)
	at or

In [4]:
spark.stop()

25/04/23 15:04:41 INFO SparkUI: Stopped Spark web UI at http://07d3ac1c3c1a:4040
25/04/23 15:04:41 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
25/04/23 15:04:41 INFO MemoryStore: MemoryStore cleared
25/04/23 15:04:41 INFO BlockManager: BlockManager stopped
25/04/23 15:04:41 INFO BlockManagerMaster: BlockManagerMaster stopped
25/04/23 15:04:41 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
25/04/23 15:04:41 INFO SparkContext: Successfully stopped SparkContext
